In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto
import torch
import time
import pandas as pd, html
from user_info import get_user_info
import requests
from tqdm.notebook import tqdm
from user_info import get_user_data_search_embed
from job_info import fetch_jobs

/home/ubuntu/infoedge/llama-recipes/env/lib/python3.10/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [2]:
from vllm import SamplingParams, LLM

sampling_params = SamplingParams(temperature=0, max_tokens=144)
llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.2")

INFO 02-26 17:26:33 llm_engine.py:73] Initializing an LLM engine with config: model='mistralai/Mistral-7B-Instruct-v0.2', tokenizer='mistralai/Mistral-7B-Instruct-v0.2', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)
INFO 02-26 17:26:48 llm_engine.py:223] # GPU blocks: 922, # CPU blocks: 2048
INFO 02-26 17:26:51 model_runner.py:394] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-26 17:26:56 model_runner.py:437] Graph capturing finished in 5 secs.


In [3]:
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",
#                                             load_in_8bit=True, 
#                                             torch_dtype=torch.float16,
#                                             device_map='auto')

# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [4]:
dataset = pd.read_csv('data/analysis_dataset.csv')

In [5]:
dataset

,job_id,application_id,user_id
0,1373681,144274567,439042
1,1373681,144261479,2015063
2,1373681,144253966,2777927
3,1373681,144241259,58130
4,1373681,144236812,2290933
...,...,...,...
136248,1376058,144442227,2151474
136249,1375030,144442249,1463255
136250,1373682,144442251,2601930
136251,1376582,144442257,1417492


In [6]:
def fetch_job_title(job_id):
    
    '''
    Returns Job Data[Hirst, Source : ElasticSearch] for the last_k days, and with flexible page size (<10,000)
    
    Input : Last-K Days, and Page result size
    
    Output : Pandas Dataframe with jobs from the last k days 
    '''
    
    # columns = ['id', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags','brandJobFlag']     #Added column name for companyStatus
    # new_cols = ['jobid', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags', 'brandJobFlag']       #Added column name for companyStatus
    
    url = "http://internal-java-job-searcher-email-backend-1607736061.ap-south-1.elb.amazonaws.com/v1/user/-777/job/jobcode?query=<JOBID>"
    url = url.replace('<JOBID>',job_id)
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    job_data = response.json()

    title = job_data[0]['title']
    jd = job_data[0]['introText']
    jd = html.unescape(jd)

    return title + '\n' + jd


In [7]:
dataset

,job_id,application_id,user_id
0,1373681,144274567,439042
1,1373681,144261479,2015063
2,1373681,144253966,2777927
3,1373681,144241259,58130
4,1373681,144236812,2290933
...,...,...,...
136248,1376058,144442227,2151474
136249,1375030,144442249,1463255
136250,1373682,144442251,2601930
136251,1376582,144442257,1417492


In [8]:
dataset['job_id'].drop_duplicates()

0         1373681
25        1373682
66        1373683
71        1373684
189       1373685
           ...   
133468    1376446
133579    1376758
134063    1375146
136145    1376759
136172    1376737
Name: job_id, Length: 2949, dtype: int64

In [9]:
dataset['user_id'].drop_duplicates()

0          439042
1         2015063
2         2777927
3           58130
4         2290933
           ...   
136201    2796578
136202    2679806
136235     986712
136245    1713212
136249    1463255
Name: user_id, Length: 30363, dtype: int64

In [10]:
dataset.sample(5)

,job_id,application_id,user_id
42539,1374535,144255170,1358359
129832,1373772,144430631,906171
130765,1375557,144432351,1651587
27175,1374136,144183656,1805375
9390,1373834,144193642,1137063


In [32]:
es_output = get_user_info(1358359)
rt = es_output['resume'][0]
rt = html.unescape(rt)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18396.07it/s]


In [33]:
answer_format = {
    "work_experience":"description of most recent work experience"
}

In [34]:
input_prompt = f'''
<s>[INST]You are an accurate AI agent working for a job platform. You will be given the raw 
unstructured text of a user's resume, Your task is to return the description of the user's
latest job experience as given in the resume. return the answer in the following format :\n{{answer_format}}\n
Return a valid JSON only
This is the resume text:\n{{resume_text}}\n
The answer is: [\INST]
'''

In [35]:
input_prompt = input_prompt.format(resume_text=rt, answer_format=answer_format)

In [36]:
outputs = llm.generate(input_prompt, sampling_params)
generated_text = outputs[0].outputs[0].text

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.07s/it]


In [37]:
generated_text

"{'work_experience': 'Currently working as a Process Specialist at Cargill since June 2021. Responsibilities include preparing Financial Statements in accordance with USGAAP and Cargill's internal guidelines, analyzing anomalies, working on periodical forecast and budget for expenses by cost centre wise, analyzing actual financial results compared with forecast and budget on monthly and quarterly basis, and providing data with trends to the commercial team. Additionally, I meet regulatory requirements by providing data to government authorities and provide cost drivers for changes in cost over months and drive monthly divisional cost governance meetings. Value-added projects include price, discount & cost analysis for effective maintenance of"

In [17]:
# start_time = time.time()
# model.eval()
# with torch.no_grad():
#     full_document = tokenizer.decode(model.generate(**model_input, max_new_tokens=32)[0], skip_special_tokens=True)

# print(f'Time taken : {time.time()-start_time}')

In [18]:
dataset.head(5)

,job_id,application_id,user_id
0,1373681,144274567,439042
1,1373681,144261479,2015063
2,1373681,144253966,2777927
3,1373681,144241259,58130
4,1373681,144236812,2290933


In [19]:
dataset.sample(frac=1)

,job_id,application_id,user_id
36786,1374412,144220143,158664
36286,1374410,144228411,1853933
32756,1374332,144230299,2793538
59461,1374912,144286093,849265
62427,1375002,144301729,84463
...,...,...,...
101756,1376152,144379743,1693198
125367,1376100,144422162,1268621
3177,1373739,144168429,1177512
31885,1374309,144216364,1702935


In [20]:
f = []
for jid in tqdm(dataset.job_id.unique()):
    sdf = fetch_jobs(str(jid))
    f.append(sdf)

jdf = pd.concat(f)

  0%|          | 0/2949 [00:00<?, ?it/s]

/tmp/ipykernel_3279/3556400985.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  jdf = pd.concat(f)


In [21]:
def get_answer_from_vllm(resume_text, job_title, llm):

    #define the input prompt here
    input_prompt = f'''<s>[INST]You are an accurate AI agent working for a job platform. You will be given the raw 
                            unstructured text of a user's resume, and the description of a job that they have applied to. Your task 
                            is to check if the resume contains prior work experience relevant to the job title being provided.
                            Respond with a JUST A SINGLE WORD answer "yes" or "no". If there's any ambiguity, answer with "no"
                            This is the job description {{job_title}}\n
                            This is the resume text:\n{{resume_text}}\n
                            The answer is: [\INST]
                            '''

    input_prompt = input_prompt.format(resume_text=resume_text,
                   job_title=jt)
    
    outputs = llm.generate(input_prompt, sampling_params)
    generated_text = outputs[0].outputs[0].text

    return generated_text

In [22]:
jdf.sample()

,id,title,introText,jobdesignation,min,max,createdBy,creatorDomainName,categoryId,jobDetailUrl,...,maxBatch,brandJobFlag,companyDomain,lableId,companyData,recruiter,jobStatusInfo,refreshStatus,location,showRecruiterProfile
0,1373765,Assistant Manager - Talent Acquisition (5-8 yrs),<p>Job Description:<br/><br/>End to end recrui...,Assiatant Manager - Talent Acquisition,5,8,197197,leadway.in,17,https://www.iimjobs.com/j/assistant-manager-ta...,...,None,0,None,None,"{'companyId': 0, 'companyName': 'Leadway Resou...","{'recruiterId': 197197, 'recruiterName': 'Aksh...",None,0,"[{'id': 7, 'name': 'Pune'}]",NaN


In [23]:
jdf[jdf['applyCount']>=70].sample()

,id,title,introText,jobdesignation,min,max,createdBy,creatorDomainName,categoryId,jobDetailUrl,...,maxBatch,brandJobFlag,companyDomain,lableId,companyData,recruiter,jobStatusInfo,refreshStatus,location,showRecruiterProfile
0,1376192,Assistant Manager - Finance & Accounts (6-9 yrs),<p>Department : Finance & Accounts</p><p><br/>...,Assistant Manager - Finance & Accounts,6,9,209485,eugenix.in,13,https://www.iimjobs.com/j/assistant-manager-fi...,...,None,0,None,None,"{'companyId': 0, 'companyName': 'Eugenix Hair ...","{'recruiterId': 209485, 'recruiterName': 'Mali...",None,1,"[{'id': 37, 'name': 'Gurgaon/Gurugram'}]",NaN


In [24]:
#filter out jobs, select ones with atleast 50 applications
count_df = dataset['job_id'].value_counts().reset_index()

In [25]:
jlist = count_df.sample(100)['job_id'].values
sample_set = dataset[dataset['job_id'].isin(jlist)]
jl2 = sample_set[['job_id','user_id']].values.tolist()

In [70]:
dataset[dataset['job_id']==1373922].shape[0]

60

In [71]:
ds = dataset[dataset['job_id']==1373922]

In [61]:
jdf[jdf['id']==1373922]

,id,title,introText,jobdesignation,min,max,createdBy,creatorDomainName,categoryId,jobDetailUrl,...,maxBatch,brandJobFlag,companyDomain,lableId,companyData,recruiter,jobStatusInfo,refreshStatus,showRecruiterProfile,location
0,1373922,EPIC Investment Partners - Finance Manager (5-...,<p><b>About Us:</b><br/><br/>EPIC Investment P...,Manager - Financial Analyst,5,8,228357,epicip.com,13,https://www.iimjobs.com/j/epic-investment-part...,...,None,0,None,None,"{'companyId': 0, 'companyName': 'EPIC Administ...","{'recruiterId': 228357, 'recruiterName': 'ROSH...",None,0,NaN,"[{'id': 38, 'name': 'Noida'}]"


In [62]:
print(html.unescape(jdf[jdf['id']==1374034]['introText'].values[0]))

<p>Role Requirements<br/><br/>- Drive the financial planning of the company by analyzing its performance and risks<br/><br/>- Retain constant awareness of the company's financial position and act to prevent problems<br/><br/>- Set up and oversee the company's finance ERP system & automation<br/><br/>- Managing all finance and accounting operations<br/><br/>- Oversee all audit and internal control operations<br/><br/>- Develop the corporate fundraising strategy and manage relationships with partners and investors<br/><br/>- Prepare timely and detailed reports on financial performance on a quarterly and annual basis<br/><br/>- Conduct analysis to make forecasts<br/><br/>- Ensure adherence to financial laws and guidelines</p>


In [63]:
ex = ds[['job_id','user_id']].to_numpy().tolist()

In [64]:
import random

In [1]:
responses = {}
# job_user_list = dataset[['job_id','user_id']].sample(frac=1).drop_duplicates().to_numpy().tolist()

# jul = random.sample(job_user_list,50)

for jobid, userid in tqdm(jl2):

    user_id_list = dataset.loc[dataset['job_id']==jobid]['user_id'].tolist()
    try:
        jt = fetch_job_title(str(jobid))
    except Exception as e:
        print(f'Error while getting job data for job id {jobid} is {e}\n')
        continue

    if jobid in responses:
        pass
    else:
        responses[jobid] = {}

    try:
        es_output = get_user_info(userid)
        rt = es_output['resume'][0]
        rt = html.unescape(rt)
    except Exception as e:
        print(f'Error while getting resume text for user_id {userid} is {e}\n')
        continue
    
    full_doc = get_answer_from_vllm(rt,jt, llm)
    responses[jobid][userid] = full_doc

In [66]:
responses

{}

In [88]:
df = get_user_data_search_embed(ds['user_id'].tolist())

def make_clickable(link):
    return f'<a href="{link}" target="_blank">{link}</a>'
    
final_usr=pd.DataFrame()

for x in df['id']:
    hy = "https://search.iimjobs.com/profile/userid"
    mini = df.loc[df['id'] == x].copy()
    mini['user_profile'] = [hy.replace("userid", str(x))]
    final_usr = pd.concat([final_usr, mini], ignore_index=True)

final_usr.reset_index(drop=True, inplace=True)
final_usr['user_profile'] = final_usr['user_profile'].apply(lambda x: make_clickable(x))

final_usr = final_usr[['id','current_designation','user_experience','user_profile']]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 54764.34it/s]


In [89]:
def make_clickable(val):
    """Convert a URL into a HTML link."""
    return f'<a href="{val}" target="_blank">{val}</a>'

final_usr.style.format({'user_profile': make_clickable})


,id,current_designation,user_experience,user_profile
0,116512,Chief Finance Officer,17,"https://search.iimjobs.com/profile/116512"" target=""_blank"">https://search.iimjobs.com/profile/116512"
1,859535,"Consultant, Technical Accounting",17,"https://search.iimjobs.com/profile/859535"" target=""_blank"">https://search.iimjobs.com/profile/859535"
2,67897,Assistant Vice President,14,"https://search.iimjobs.com/profile/67897"" target=""_blank"">https://search.iimjobs.com/profile/67897"
3,1292574,Business Finance Head- Assistant Vice President,16,"https://search.iimjobs.com/profile/1292574"" target=""_blank"">https://search.iimjobs.com/profile/1292574"
4,112718,Assistant General Manager (AGM),11,"https://search.iimjobs.com/profile/112718"" target=""_blank"">https://search.iimjobs.com/profile/112718"
5,207046,Senior Finance Manager,16,"https://search.iimjobs.com/profile/207046"" target=""_blank"">https://search.iimjobs.com/profile/207046"
6,946806,Deputy Director,8,"https://search.iimjobs.com/profile/946806"" target=""_blank"">https://search.iimjobs.com/profile/946806"
7,147544,CFO,13,"https://search.iimjobs.com/profile/147544"" target=""_blank"">https://search.iimjobs.com/profile/147544"
8,58130,Head - Treasury & Operations (part of Senior Management Team),16,"https://search.iimjobs.com/profile/58130"" target=""_blank"">https://search.iimjobs.com/profile/58130"
9,41321,Director Corporate Strategy and FP&A,9,"https://search.iimjobs.com/profile/41321"" target=""_blank"">https://search.iimjobs.com/profile/41321"


## Running loop for applications

In [58]:
responses = {}

In [59]:
random_jobs_list = dataset['job_id'].sample(100).values.tolist()

In [60]:
job_user_list = dataset[dataset['job_id'].isin(random_jobs_list)][['job_id','user_id']]

In [69]:
len(responses.keys())

84

In [105]:
r2 = {}
r3 = {}

for keys, vals in responses.items():
    extracted_data = {key: value.strip().split('\n')[0] for key, value in vals.items()}
    r2[keys] = extracted_data

    total_count = len(extracted_data)
    no_count = sum(1 for response in extracted_data.values() if response.lower().startswith('no'))
    
    # Calculating the percentage of "no" responses
    try:
        no_percentage = (no_count / total_count) * 100
        r3[keys] = no_percentage
    except Exception as e:
        print(keys)

In [106]:
len(responses.keys())

1

In [107]:
responses

{1373922: {900486: " yes\n                            ]\n\nThe user's resume shows a significant amount of experience in investment banking, corporate finance, private equity, and credit underwriting",
  27740: ' yes\n                            ]\n\nThe resume contains prior work experience relevant to the job title "Finance Manager" with 5-8 years of experience.',
  2088388: " yes.\n                            The user's resume shows 8 years of experience in deal advisory services, including experience in financial modeling, valuation, and",
  2804905: "no.\n                            The user's resume does not mention any prior work experience relevant to the Finance Manager job description provided. The resume mentions experience as",
  2800387: 'no.\n                            The resume does not mention any experience in investment banking, corporate finance, private equity, or investment management, which are the key areas mentioned',
  839460: ' yes\n                         

In [79]:
import numpy as np
fls = np.array(list(r3.values()))

In [81]:
fls.mean()

10.397872300140472